In [2]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [3]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 28:31].values
classes = np.reshape(classes, (2000, 700, 3))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 3)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNLSTM, Bidirectional, Masking

In [5]:
def criarRede():
    model = Sequential()
  
    #model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
  
    model.add(Dense(3, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [6]:
accu = []
precisions = []
recalls = []
f1 = []

In [7]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 3))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 3))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accu.append(balanced_accuracy_score(y_test, predicted)) 
    precisions.append(precision_score(y_test, predicted, average = 'weighted'))
    recalls.append(recall_score(y_test, predicted, average = 'weighted'))
    f1.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0819 22:51:08.051712 13068 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0819 22:51:08.057696 13068 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0819 22:51:08.058693 13068 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   0    1    2 ... 1996 1998 1999] TEST: [  10   12   23   26   29   30   36   37   38   39   41   46   61   68
   78   79   82   83   85   90   95   97  104  107  117  119  124  125
  131  143  144  155  160  172  174  186  193  201  206  207  220  238
  246  254  255  264  268  269  275  283  285  289  297  299  309  316
  317  320  331  332  347  349  350  353  354  357  359  360  366  372
  376  379  383  389  391  399  401  403  407  413  414  429  430  432
  433  442  443  445  446  450  454  457  458  470  475  483  486  495
  507  510  512  523  525  528  532  533  537  538  542  545  549  551
  561  563  566  568  571  574  578  579  583  585  587  594  601  604
  605  608  610  613  615  620  624  625  628  630  632  639  643  657
  661  664  665  666  680  681  687  691  699  700  711  713  714  725
  726  730  743  751  755  757  760  764  768  773  775  777  781  790
  793  796  798  802  807  815  827  832  834  839  844  849  851  852
  853  864  867  870  871  8

W0819 22:51:09.385150 13068 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3591 - acc: 0.7556
Epoch 2/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.3205 - acc: 0.8511
Epoch 3/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.3067 - acc: 0.8600
Epoch 4/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2997 - acc: 0.8639
Epoch 5/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2970 - acc: 0.8649
Epoch 6/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2930 - acc: 0.8672
Epoch 7/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2908 - acc: 0.8676
Epoch 8/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2880 - acc: 0.8691
Epoch 9/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2864 - acc: 0.8703
Epoch 10/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2834 - a

1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2051 - acc: 0.9068
Epoch 36/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2068 - acc: 0.9051
Epoch 37/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.1984 - acc: 0.9095
Epoch 38/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.1935 - acc: 0.9119
Epoch 39/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.1931 - acc: 0.9123
Epoch 40/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.1868 - acc: 0.9153
Epoch 41/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.1845 - acc: 0.9161
Epoch 42/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.1804 - acc: 0.9178
Epoch 43/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.1765 - acc: 0.9197
Epoch 44/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.1747 - acc: 

Epoch 1/100
1600/1600 [==============================] - 12s 7ms/sample - loss: 0.3579 - acc: 0.7968
Epoch 2/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.3233 - acc: 0.8497
Epoch 3/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.3068 - acc: 0.8611
Epoch 4/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.3010 - acc: 0.8645
Epoch 5/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2974 - acc: 0.8659
Epoch 6/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2943 - acc: 0.8668
Epoch 7/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2898 - acc: 0.8693
Epoch 8/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2889 - acc: 0.8696
Epoch 9/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2851 - acc: 0.8716
Epoch 10/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2822 - a

1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2334 - acc: 0.8988
Epoch 36/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2313 - acc: 0.9001
Epoch 37/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2289 - acc: 0.9008
Epoch 38/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2261 - acc: 0.9024
Epoch 39/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2216 - acc: 0.9046
Epoch 40/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2222 - acc: 0.9043
Epoch 41/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2177 - acc: 0.9063
Epoch 42/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2158 - acc: 0.9075
Epoch 43/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2132 - acc: 0.9084
Epoch 44/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2099 - acc: 

Epoch 1/100
1600/1600 [==============================] - 12s 7ms/sample - loss: 0.3662 - acc: 0.7987
Epoch 2/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.3341 - acc: 0.8447
Epoch 3/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.3141 - acc: 0.8582
Epoch 4/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.3078 - acc: 0.8619
Epoch 5/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.3027 - acc: 0.8648
Epoch 6/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.3002 - acc: 0.8654
Epoch 7/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2961 - acc: 0.8673
Epoch 8/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2941 - acc: 0.8682
Epoch 9/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2926 - acc: 0.8683
Epoch 10/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.2928 - a

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_2 (Masking)          (None, 700, 20)           0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 700, 200)          96800     
_________________________________________________________________
dropout_5 (Dropout)          (None, 700, 200)          0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 700, 200)          240800    
_________________________________________________________________
dropout_6 (Dropout)          (None, 700, 200)          0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 700, 200)          240800    
_________________________________________________________________
dropout_7 (Dropout)          (None, 700, 200)          0         
__________

In [4]:
print('Acurácias total')
print(accu)
accu = np.array(accu)
print(accu.mean())
print('Precision total')
print(precisions)
precisions = np.array(precisions)
print(precisions.mean())
print('Recalls total')
print(recalls)
recalls = np.array(recalls)
print(recalls.mean())
print('F1 total')
print(f1)
f1 = np.array(f1)
print(f1.mean())

Acurácias total
0.8019385058318249
Precision total
0.8124853288891728
Recalls total
0.8124986326813181
F1 total
0.8119622556029673
